In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand
147,AFG,Asia,Afghanistan,2020-06-05,18054,787,300,6,463.775,20.217,...,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50
236,ALB,Europe,Albania,2020-06-05,1197,13,33,0,415.943,4.517,...,13.188,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89
389,DZA,Africa,Algeria,2020-06-05,9831,205,681,14,224.191,4.675,...,6.211,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-04,France,151183.0,0.0,29124.0,0.0,0.0,0.0
0,2020-06-04,China,84219.0,0.0,4652.0,0.0,0.0,0.0
0,2020-06-04,Italy,234280.0,0.0,33676.0,0.0,0.0,0.0
0,2020-06-04,Spain,240668.0,0.0,27965.0,0.0,0.0,0.0
0,2020-06-04,United States,1874436.0,0.0,108330.0,0.0,0.0,0.0
0,2020-06-04,World,6599609.0,0.0,391358.0,0.0,0.0,0.0
0,2020-06-04,United Kingdom,281888.0,0.0,40086.0,0.0,0.0,0.0
0,2020-06-04,Germany,183184.0,0.0,8608.0,0.0,0.0,0.0
0,2020-06-04,Iran,163709.0,0.0,8081.0,0.0,0.0,0.0
0,2020-06-04,Turkey,167181.0,0.0,4629.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-05,France,151183.0,152444.0,29124.0,29065.0,-1261.0,59.0
0,2020-06-05,China,84219.0,84171.0,4652.0,4638.0,48.0,14.0
0,2020-06-05,Italy,234280.0,234013.0,33676.0,33689.0,267.0,-13.0
0,2020-06-04,Spain,240668.0,240660.0,27965.0,27940.0,8.0,25.0
0,2020-06-05,United States,1874436.0,1872660.0,108330.0,108211.0,1776.0,119.0
0,2020-06-05,World,6599609.0,6603329.0,391358.0,391732.0,-3720.0,-374.0
0,2020-06-05,United Kingdom,281888.0,281661.0,40086.0,39904.0,227.0,182.0
0,2020-06-05,Germany,183184.0,183271.0,8608.0,8613.0,-87.0,-5.0
0,2020-06-05,Iran,163709.0,164270.0,8081.0,8071.0,-561.0,10.0
0,2020-06-05,Turkey,167181.0,167410.0,4629.0,4630.0,-229.0,-1.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")